In [ ]:
import subprocess
import os
from trl import SFTConfig, SFTTrainer, ModelConfig, get_kbit_device_map, get_peft_config, get_quantization_config
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value


dataset_train = load_dataset("stanfordnlp/imdb", split="train")



In [2]:
BASE_MODEL = "Qwen/Qwen2-0.5B"
torch.cuda.empty_cache()

model_args = ModelConfig(
    model_name_or_path=BASE_MODEL,
    trust_remote_code=True,
    torch_dtype='auto',
)

training_args = SFTConfig(
    output_dir='/root/autodl-tmp/SFT_imdb',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_steps=100,
    report_to = 'none',
    push_to_hub=True,
    hub_model_id='august66/qwen2-sft-final',
    save_steps=3000,         
    save_total_limit=3,     
    save_strategy="steps", 
)

quantization_config = get_quantization_config(model_args)

model_kwargs = dict(
    revision = model_args.model_revision,
    trust_remote_code=model_args.trust_remote_code,
    torch_dtype=model_args.torch_dtype,
    use_cache = False if training_args.gradient_checkpointing else None,
    device_map=get_kbit_device_map() if quantization_config is not None else None,
    quantization_config=quantization_config,
)

training_args.model_init_kwargs = model_kwargs
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=model_args.trust_remote_code,
    use_fast=True,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token





tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [3]:
trainer = SFTTrainer(
    model = model_args.model_name_or_path,
    args = training_args,
    train_dataset = dataset_train,
    processing_class = tokenizer,
    peft_config=get_peft_config(model_args)
) 
trainer.train()# ✅ manually save final model

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Converting train dataset to ChatML:   0%|          | 0/25000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

Step,Training Loss
100,3.238900
200,3.185600
300,3.236400
400,3.208500
500,3.180300
600,3.204900
700,3.180200
800,3.173900
900,3.166800
1000,3.153100


TrainOutput(global_step=9375, training_loss=3.0734153483072917, metrics={'train_runtime': 2976.7233, 'train_samples_per_second': 25.195, 'train_steps_per_second': 3.149, 'total_flos': 1.0690613019923866e+17, 'train_loss': 3.0734153483072917})

In [4]:
repo_id = "august66/qwen2-sft-final"
# 2) Push model weights & config to HF Hub
trainer.model.push_to_hub(repo_id, safe_serialization=True)

# 3) Push tokenizer files (vocab, merges, etc.)
trainer.tokenizer.push_to_hub(repo_id)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/august66/qwen2-sft-final/commit/d80b121c3c21e7bef5a0e56f23c2d345fdf9ee48', commit_message='Upload tokenizer', commit_description='', oid='d80b121c3c21e7bef5a0e56f23c2d345fdf9ee48', pr_url=None, repo_url=RepoUrl('https://huggingface.co/august66/qwen2-sft-final', endpoint='https://huggingface.co', repo_type='model', repo_id='august66/qwen2-sft-final'), pr_revision=None, pr_num=None)